In [1]:
!pip install albumentations


In [2]:
import albumentations as A
import cv2
from pathlib import Path
import os
from tqdm import tqdm

In [3]:
transform = A.Compose([
    A.Rotate(limit=15, p=0.5),  # Rotar la imagen ±15° con una probabilidad de 0.5
    A.RandomScale(scale_limit=0.1, p=0.5),  # Escalar la imagen hasta un 10% más grande o más pequeña
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),  # Aplicar traslación, escalado y rotación al mismo tiempo
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),  # Ajustar el brillo y el contraste
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),  # Agregar ruido gaussiano
    A.HorizontalFlip(p=0.5),  # Voltear horizontalmente con probabilidad 0.5
    A.VerticalFlip(p=0.5)  # Voltear verticalmente con probabilidad 0.5
])

In [7]:
def augment_and_save_images(input_dir, output_dir, numaug=10):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Procesar cada subcarpeta en el directorio de entrada
    for class_folder in ['correcto', 'incorrecto']:
        class_path = input_dir / class_folder
        output_class_path = output_dir / class_folder
        output_class_path.mkdir(parents=True, exist_ok=True)

        # Procesar cada imagen en la carpeta de clase
        for image_path in tqdm(list(class_path.glob('*'))):
            # Leer la imagen
            image = cv2.imread(str(image_path))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Guardar la imagen original en la nueva carpeta
            cv2.imwrite(str(output_class_path / image_path.name), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

            # Generar y guardar las imágenes aumentadas
            for i in range(numaug):
                augmented = transform(image=image)['image']
                augmented_image_path = output_class_path / f"{image_path.stem}_aug_{i}{image_path.suffix}"
                cv2.imwrite(str(augmented_image_path), cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))

In [8]:
augment_and_save_images('/content/drive/MyDrive/behavior2/dataset/dataset1', '/content/drive/MyDrive/behavior2/dataset/dataset_aug')

100%|██████████| 75/75 [04:05<00:00,  3.27s/it]
